# Marine Ice Sheet Stability Demo

This is a demonstration of some models for the evolution of marine ice sheets. This was modified for the Princeton AOS workshop 2021. Last updated by Alex Robel on July 20, 2021.

## Load Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

## Define simple model with plotting

In [4]:
def plotmodel(accum,oceanmelt,Linit):
    #other parameters
    rho_i = 917    #ice density
    rho_w = 1028   #water density
    g = 9.81       #gravity (in m/s^2)
    beta = 4       #GL flux exponent

    #time stepping
    tf = 5000       #length of simulation
    nt = 5000       #number of time steps
    dt = tf/nt     #length of time step
    L = np.zeros([nt+1]) #pre-allocate volume vector
    L[0] = Linit     #initial condition
    
    omega = oceanmelt*5e-7   #GL flux coefficient
    #run model for grounding line Evolution
    for t in range(nt):
        b = -(729 - 2184.8*(L[t]/750e3)**2 + 1031.72*(L[t]/750e3)**4 - 151.72*(L[t]/750e3)**6)   #calculate bed depth at current grounding line position from MISMIP topography
        hg = (rho_w/rho_i)*b      #calculate flotation thickness at GL
        Q_g = omega*(hg**beta)      #calculate GL flux
  
        dL_dt = (accum*L[t] - Q_g)/hg #calculate RHS of model
    
        L[t+1] = L[t] + dL_dt*dt       #use forward euler to advance model
    
    #calculate some diagnostics
    time = np.linspace(0,tf,nt+1)
    x = np.linspace(400.0,1500.0,1000)
    bx = (729 - 2184.8*(x/750)**2 + 1031.72*(x/750)**4 - 151.72*(x/750)**6)
    binit = (729 - 2184.8*(L[0]/750e3)**2 + 1031.72*(L[0]/750e3)**4 - 151.72*(L[0]/750e3)**6)
    bend = (729 - 2184.8*(L[-1]/750e3)**2 + 1031.72*(L[-1]/750e3)**4 - 151.72*(L[-1]/750e3)**6)
    hgs = -(rho_w/rho_i)*(729 - 2184.8*(x/750)**2 + 1031.72*(x/750)**4 - 151.72*(x/750)**6)
    hgsl = -(rho_w/rho_i)*(729 - 2184.8*(L/750e3)**2 + 1031.72*(L/750e3)**4 - 151.72*(L/750e3)**6)

    #plot time evolution of grounding line
    fig, (ax1, ax2, ax3) = plt.subplots(3,figsize=(10,10))

    ax1.set_xlabel('x (km)')  # we already handled the x-label with ax1
    ax1.set_ylabel('Bed topography (m)')  # we already handled the x-label with ax1
    ax1.plot(x, bx)
    ax1.plot(L[0]/1e3,binit,'ks', markersize=12)
    ax1.plot(L[-1]/1e3,bend,'ko', markersize=12)
    
    ax2.set_ylabel('Grounding line position (km)')  # we already handled the x-label with ax1
    ax2.set_xlabel('Time (years)')  # we already handled the x-label with ax1
    ax2.plot(time, L/1e3)
    
    ax3.set_ylabel('Fluxes (m^2/yr)')  # we already handled the x-label with ax1
    ax3.set_xlabel('x (km)')  # we already handled the x-label with ax1
    ax3.plot(x, accum*x*1e3,'b')
    ax3.plot(x, omega*(hgs**beta),'r')
    ax3.plot(L[0]/1e3, accum*L[0],'bs', markersize=12)
    ax3.plot(L[0]/1e3, omega*(hgsl[0]**beta),'rs', markersize=12)
    ax3.plot(L[-1]/1e3, accum*L[-1],'bo', markersize=12)
    ax3.plot(L[-1]/1e3, omega*(hgsl[-1]**beta),'ro', markersize=12)
    plt.ylim(0,10e5)
    plt.xlim(500,1600)
    
    plt.show()
    
    plt.show()

## Generate interactive plot of simple model evolution

In [14]:
interactive_plot = interactive(plotmodel, accum=(0.05,0.7,0.05), oceanmelt=(0.5,2,0.1),Linit=(600e3,1500e3,100e3))
interactive_plot

interactive(children=(FloatSlider(value=0.35000000000000003, description='accum', max=0.7, min=0.05, step=0.05…

## Instructions for break-out groups 

Split into groups of 2-3 (via breakout rooms), and answer the following questions:
1. Lower accumulation rate via snowfall (using top slider). Describe what happens to the transient response, and the flux balance. Is there a tipping point of accumulation at which things change?
2. Now (with accumulation back to 0.3 m/yr) progressively increase oceanmelt (which increases the grounding line flux). What changes?
3. Finally, do the above experiments again, but with an initial grounding line position (Li